In [1]:
import time
import os

import torch
import torch.nn as nn
import clip
import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms
device = "cuda" if torch.cuda.is_available() else "cpu"
ClipModel, transform = clip.load("ViT-B/32", device)
from model.robot_pose_prediction import RobotPosePrediction,preprocess
model=RobotPosePrediction(ClipModel,3)

import sys
sys.path.append("../")
# import utils
from utils import *
import GrabSim_pb2_grpc
import GrabSim_pb2

secen = SceneManager.SceneManager()
secen.load_scene(map_id=3,scene_num=1)
time.sleep(5)
camera = CameraController.CameraController(secen)
navigator= NavigationController.NavigationController(secen)
agent = secen.get_scene_info()
frame_count=0

while True:        
    walk_value = [agent.location.X,agent.location.Y, agent.rotation.Yaw]#机器人的当前位姿
    img_data = camera.capture_image(GrabSim_pb2.CameraName.Head_Color,0)
    img = img_data.images[0]
    img = np.frombuffer(img.data, dtype=img.dtype).reshape((img.height, img.width, img.channels))
    instr = "we have three person and want to sit near the window."#"get me the chair."
    img,instr,state=preprocess(image=img,instruction=instr,state=walk_value)
    move_pose = model(img.to(device),instr.to(device),state.to(device)).tolist()
    # dx,dy,dz=random.random()*500,random.random()*500,random.random()*500
    dx,dy=move_pose[0]
    print(f"walk_v for this frame:,{dx},{dy}")       
    action = navigator.navigate_to_limit(x=agent.location.X+dx*100,y=agent.location.Y+dy*100,yaw=agent.rotation.Yaw - 90,velocity=900,distance_limit=100)
    print("navi result:",action)  # print navigation info
    print(agent.info)  # print navigation info
    time.sleep(2)
    frame_count+=1


c:\Users\12245\miniconda3\envs\HarixSim\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


maps {
  name: "Empty"
  width: 2500.0
  height: 2500.0
}
maps {
  name: "Desk"
  width: 600.0
  height: 300.0
}
maps {
  name: "Room"
  width: 2500.0
  height: 2500.0
}
maps {
  name: "Starbucks"
  width: 1400.0
  height: 2800.0
}
maps {
  name: "TG"
  width: 3200.0
  height: 9600.0
}
maps {
  name: "NursingRoom"
  width: 6000.0
  height: 3000.0
}
maps {
  name: "QiYeHao"
  width: 1500.0
  height: 1000.0
}
maps {
  name: "TLV2"
  width: 300.0
  height: 320.0
}

walk_v for this frame:,0.3570035696029663,0.9341030716896057
navi result: location {
  X: 247.0
  Y: 500.0
}
rotation {
  Yaw: 180.0
}
joints {
  name: "Knee_X"
  location {
    X: 246.98728942871094
    Y: 500.0
    Z: 43.66059494018555
  }
  angle: -1.9083340593514797e-14
}
joints {
  name: "Back_Z"
  location {
    X: 246.97854614257812
    Y: 500.0
    Z: 73.71397399902344
  }
  angle: 6.830189704487566e-06
}
joints {
  name: "Back_X"
  location {
    X: 246.9785614013672
    Y: 500.0
    Z: 86.15656280517578
  }
  angle: -

KeyboardInterrupt: 